#### Génération d'un index "point" spatial sous Neo4j pour tous les objets

Objectifs :

- Génération d'un index spatial pour tous les objets CITY, ROAD_POINT, et NATIONAL ROAD_POINT

In [1]:
import geopandas as gpd
import pandas as pd
import geopy.distance
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import copy
import joblib

from neo4j import GraphDatabase, basic_auth

In [2]:
def create_spatial_index(tx, entity):

    query = "  MATCH (n1:" + entity + ") \
            SET n1.location = point({longitude:n1.x, latitude:n1.y}) \
            RETURN count(n1) AS count"

    result = tx.run(query)
    return result.data()

In [3]:
def change_type_string_float(tx, entity):

    query = "  MATCH (n1:" + entity + ") \
            SET n1.x = toFloat(n1.x) \
            SET n1.y = toFloat(n1.y) \
            RETURN count(n1) AS count"

    result = tx.run(query)
    return result.data()

In [4]:
def create_index_road_point(tx):
    query = "  CREATE INDEX road_index \
            FOR (n:ROAD_POINT) \
            ON (n.x, n.y) "

    result = tx.run(query)
    return result.data()

In [8]:
def create_spatial_index_national_point(tx):

    query = "  MATCH ()-[r:LINKED_TO]-() \
            SET r.location = point({longitude:r.x_1, latitude:r.y_1}) \
            RETURN count(r) AS count"

    result = tx.run(query)
    return result.data()

In [11]:
def create_spatial_index_national_point_2(tx):

    query = "  MATCH ()-[r:LINKED_TO]-() \
            SET r.location_2 = point({longitude:r.x_2, latitude:r.y_2}) \
            RETURN count(r) AS count"

    result = tx.run(query)
    return result.data()

In [13]:
def create_spatial_index_national_point_center(tx):

    query = "  MATCH ()-[r:LINKED_TO]-() \
            SET r.location_center = point({longitude:r.x_center, latitude:r.y_center}) \
            RETURN count(r) AS count"

    result = tx.run(query)
    return result.data()

In [5]:
PASSWORD_NEO4J = 'passwordneo4j'

In [6]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", PASSWORD_NEO4J))

Add index for communes :

In [ ]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_spatial_index, 'CITY')

driver.close()

In [7]:
nb_nodes = result[0]['count']
                
print(f'Total Numbers of index created : {nb_nodes}')

Total Numbers of index created : 34460


Add index for road_point :

Before that, we must convert x and y coords to float...

In [14]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(change_type_string_float, 'ROAD_POINT')

driver.close()

nb_points = result[0]['count']
                
print(f'Total Numbers of index created : {nb_points}')

Total Numbers of index created : 523308


In [15]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_spatial_index, 'ROAD_POINT')

driver.close()

nb_points = result[0]['count']
                
print(f'Total Numbers of index created : {nb_points}')

Total Numbers of index created : 523308


Moreover, We must create index for road points...

In [9]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_spatial_index_national_point)

driver.close()

In [10]:
result

[{'count': 11916}]

In [12]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_spatial_index_national_point_2)

driver.close()

In [14]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_spatial_index_national_point_center)

driver.close()